In [1]:
import arcgis
from collections import defaultdict

# Supporting functions and class

In [6]:
def get_users_folders_and_items(gis, username):
    """Create a dictionary containing every folder and a list of references to its items
    """
    user = gis.users.get(username)
    user_folders = [folder['title'] for folder in user.folders]
    user_folders.append(None)
    
    folders_and_items = {}
    for folder in user_folders:
        folders_and_items[folder] = []
        for item in user.items(folder, 1000):
            folders_and_items[folder].append(item)
        folders_and_items[folder].sort(key=lambda x:x.type)
    
    return folders_and_items

In [7]:
def print_items(folders_and_items):
    """Pretty formatting for a dictionary of folders and their items 
    """
    item_total = 0
    item_type_counts = defaultdict(int)
    for folder, items in folders_and_items.items():
        print(folder)
        print('='*15)
        for item in items:
            print(f'{item.title:<50} {item.type:>30}')
            item_type_counts[item.type] += 1
        item_total += len(items)
        print()
    
    item_type_counts = dict(sorted(item_type_counts.items(), key=lambda item:item[1]))
    
    print('Totals:')
    print('='*15)
    print(f'Folders: {len(folders_and_items)}')
    print(f'Items: {item_total}')
    print()
    for item_type, count in item_type_counts.items():
        print(f'{item_type:<20} {count:>30}')
        
    

In [8]:
def clone_folder(target_gis, folder_name, items_in_folder):
    """Clones a folder's items, attempting to do most complex items first (apps -> maps -> services -> everything else). Will
    skip any items that error out and move on to the next.
    
    target_gis: The AGOL org the content should be cloned into
    folder_name: The folder name (as a str) to clone
    items_in_folder: List of item objects associated with folder_name
    
    Returns:
    cloned_items: List of all item objects returned by target_gis.content.clone_items() (will include any dependencies of the 
                  actual item being cloned)
    errored_items: Any items from items_in_folder that raised an error during copy (which could be caused by the item itself
                   or any of its dependnecies)
    """
    
    #: hold officially cloned items to return for later review
    cloned_items = []
    
    #: items that error out
    errored_items = []
    
    #: Pre-populate the types list with complex types so that they clone any dependancies along the way
    types = ['Web Mapping Application', 'Web Map', 'Feature Service']
    
    #: Translate our list of items into a dictionary based on item.type
    items_by_type = defaultdict(list)
    for item in items_in_folder:
        items_by_type[item.type].append(item)
        
        #: Add the type to our list of types
        if item.type not in types:
            types.append(item.type)
    
    #: Use list of types to clone items in desired order (complex -> simple)
    for item_type in types:
        if item_type in items_by_type:
            for item in items_by_type[item_type]:
                print(item)
                try:
                    cloned_items.extend(target_gis.content.clone_items([item], folder=folder_name))
                except Exception as e:
                    print(e)  #: Watch this to see what item caused the error; often caused by depenancies.
                    errored_items.append(item)
    
    return cloned_items, errored_items

In [9]:
class CloningTank:
    """Clones folders and keeps track of items by success/failure. A folder can have individual items error out without
    having 'failed'. 
    """

    def __init__(self, username):
        self.username = username
        self.folders_and_items = None
        self.completed_folders = []
        self.remaining_folders = []
        self.errored_items_by_folders = defaultdict(list)
        self.cloned_items_by_folder = defaultdict(list)
        
    def get_items(self, source_gis):
        """Load items by folder from source_gis only if there are no existing folders/items.
        """
        
        self.folders_and_items = get_users_folders_and_items(source_gis, self.username)
        if not self.remaining_folders:
            self.remaining_folders.extend(self.folders_and_items.keys())
        else:
            print('self.remaining_folders not empty; not modified')
        
    def print_users_items(self):
        print_items(self.folders_and_items)
        
    def clone_folders(self, target_gis, foldernames):
        """Clone foldernames into target_gis. foldernames can either be a single str or a list of str's.
        """
        
        if isinstance(foldernames, str):
            foldernames = [foldernames]
        
        #: Don't try to clone finished folders. Duplication in the target_gis doesn't seem to be an issue, but not spending
        #: time on completed folders may speed things up.
        for foldername in foldernames:
            if foldername in self.completed_folders:
                print(f'{foldername} already cloned; skipping')
                continue

            try:
                self.cloned_items_by_folder[foldername], self.errored_items_by_folders[foldername] = clone_folder(target_gis, foldername, self.folders_and_items[foldername])
            
            #: Any errors in cloning items via clone_items() should have been properly handled by clone_folder(); this 
            #: will catch any other errors that may occur and will stop the folder from continuing.
            except Exception as e:
                print(e)
                
            #: If (and only if) try succeeds, move the folder from remaining to completed.
            else:  
                self.completed_folders.append(foldername)
                self.remaining_folders.remove(foldername)
                
    def print_cloned_items_by_folder(self):
        print_items(self.cloned_items_by_folder)
        
    def print_errored_items_by_folder(self):
        print_items(self.errored_items_by_folders)


# Cloning Process

In [ ]:
#: Set up source 
source_user = 'UtahAGRC'
source_url = 'https://utah.maps.arcgis.com'
source_gis = arcgis.gis.GIS(source_url, source_user)

#: Set up target. 
#: _admin can be any org admin or the user whose content you want to clone.
destination_admin = 'username'
destination_url = 'https://someonesomething.maps.arcgis.com'
destination_gis = arcgis.gis.GIS(destination_url, destination_admin)
#: The user whose data you want to clone; may or may not be the same as the admin.
users_data_to_clone = 'username'


In [ ]:
#: Create a CloningTank for our user and get their items.
user = CloningTank(users_data_to_clone)
user.get_items(source_gis)


In [ ]:
#: Can either specify the folder(s) to copy, or just pass user.remanining_folders (initially populated by get_items())
user.clone_folders(destination_gis, 'Folder Name')
user.clone_folders(destination_gis, ['Folder one', 'Folder two'])
user.clone_folders(destination_gis, user.remaining_folders)

In [ ]:
#: The root folder (None) is added to the list of folders as part of get_items(). To clone individually, you have to pass a 
#: list containing None:
user.clone_folders(destination_gis, [None])

In [ ]:
#: If you need to try to clone a folder a second time, you'll have to manually remove it from user.completed_folders
#: before cloning it again (.clone_folders() won't clone a folder that's in .completed_folders)
user.completed_folders.remove('Folder one')
user.clone_folders(destination_gis, 'Folder one')

In [ ]:
#: Print formatted info about the folders
user.print_users_items()  #: All the source folders and items
user.print_cloned_items_by_folder()  #: All the items cloned via clone_items(); will contain any dependent items 
user.print_errored_items()  #: Any parent items that failed to clone due to errors in itself or any of its dependents